# Data manipulation
includes importing the data and feature engineering

In [ ]:
#Library for importing csv files
import pandas as pd
#Library for matrix operations
import numpy as np
#Set random seeds to ensure that the model can be reproduced
import random
np.random.seed(42)
random.seed(42)

## importing training data and test data

In [ ]:
# importing data which is in .csv format
train_df=pd.read_csv("train_df.csv")
print(f"len(train_df):{len(train_df)}")
test_df=pd.read_csv("test_df.csv")
print(f"len(test_df):{len(test_df)}")

len(train_df):263160
len(test_df):17136


In [ ]:
# concatinating the training and test data frames for futher data manipulation
total_df=pd.concat((train_df,test_df),axis=0)
# dropping f1 and f2 columns in the dataset since it is not informative
total_df.drop(["f1","f2"], axis=1,inplace= True)

In [ ]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 280296 entries, 0 to 17135
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            280296 non-null  int64  
 1   id_encode     280295 non-null  float64
 2   hour          280295 non-null  float64
 3   parking_free  280295 non-null  float64
 4   year          280295 non-null  float64
 5   month         280295 non-null  float64
 6   day           280295 non-null  float64
 7   power         263159 non-null  float64
dtypes: float64(7), int64(1)
memory usage: 19.2 MB


In [ ]:
#forward_fill: Missing values ​​are filled according to the previous value
total_df.fillna(method='ffill', inplace=True)
total_df['ds']=pd.to_datetime(total_df[['year','month','day','hour']])
total_df["Day sin"] = np.sin(total_df["day"]*(2*np.pi / 30))
total_df["Day cos"] = np.cos(total_df["day"]*(2*np.pi / 30))
total_df['sin month']=np.sin(2*np.pi*total_df['month']/12)
total_df['cos month']=np.cos(2*np.pi*total_df['month']/12)
total_df['sin_hour']=np.sin(2*np.pi*total_df['hour']/24)
total_df['cos_hour']=np.cos(2*np.pi*total_df['hour']/24)
total_df.drop(columns= ["day", "month","hour"], axis=1, inplace=True)

In [ ]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 280296 entries, 0 to 17135
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   id            280296 non-null  int64         
 1   id_encode     280296 non-null  float64       
 2   parking_free  280296 non-null  float64       
 3   year          280296 non-null  float64       
 4   power         280296 non-null  float64       
 5   ds            280296 non-null  datetime64[ns]
 6   Day sin       280296 non-null  float64       
 7   Day cos       280296 non-null  float64       
 8   sin month     280296 non-null  float64       
 9   cos month     280296 non-null  float64       
 10  sin_hour      280296 non-null  float64       
 11  cos_hour      280296 non-null  float64       
dtypes: datetime64[ns](1), float64(10), int64(1)
memory usage: 27.8 MB


In [ ]:
total_df.drop(columns=["id", "year", "ds", "parking_free"], inplace= True)
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 280296 entries, 0 to 17135
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id_encode  280296 non-null  float64
 1   power      280296 non-null  float64
 2   Day sin    280296 non-null  float64
 3   Day cos    280296 non-null  float64
 4   sin month  280296 non-null  float64
 5   cos month  280296 non-null  float64
 6   sin_hour   280296 non-null  float64
 7   cos_hour   280296 non-null  float64
dtypes: float64(8)
memory usage: 19.2 MB


## normalizing the power column
Normalizing using the Standard Deviation Normalization method

In [ ]:
total_df["power"] = (total_df["power"] - total_df["power"].mean()) / total_df["power"].std()

In [ ]:
# total_df['up_of_month']=(total_df['day']<=10)
# total_df['down_of_month']=(total_df['day']>20)
# total_df['morning']=(total_df['hour']>5)&(total_df['hour']<=12)
# total_df['afternoon']=(total_df['hour']>12)&(total_df['hour']<=19)
# total_df['evening']=1-total_df['morning']-total_df['afternoon']

# total_df['ds']=pd.to_datetime(total_df[['year','month','day','hour']])
# total_df['weekday']=total_df['ds'].dt.weekday
# total_df['sin_week']=np.sin(2*np.pi*total_df['weekday']/7)
# total_df['cos_week']=np.cos(2*np.pi*total_df['weekday']/7)
# total_df['is_friday']=(total_df['weekday']==4)
# total_df['is_weekend']=(total_df['weekday']==5)|(total_df['weekday']==6)
# total_df['day_of_year']=total_df['ds'].dt.dayofyear

# #GetChina's holidays
# # holiday = holidays.China()
# # total_df['ds']=pd.to_datetime(total_df[['year','month','day','hour']])
# # ds=total_df['ds'].values
# # is_holiday = [0 if holiday.get(pd.to_datetime(ds[i]))=="None" else 1 for i in range(len(ds))]
# # total_df['is_holiday']=is_holiday

# total_df['sin_hour']=np.sin(2*np.pi*total_df['hour']/24)
# total_df['cos_hour']=np.cos(2*np.pi*total_df['hour']/24)


# total_df.drop(['ds'],axis=1,inplace=True)
# total_df.head()

In [ ]:
# train_df=total_df[:len(train_df)]
# keys=train_df.keys()
# for key in keys:
#     values=np.unique(train_df[key].values)#Get the value of each column

#     if len(values)<=500 and key!="power":
#         #print(f"key:{key},values:{values}")
#         key_target=train_df['power'].groupby([train_df[key]]).mean()
#         keys=key_target.keys().values
#         target=key_target.values
#         key_target=pd.DataFrame({key:keys,key+"_target_mean":target})
#         total_df=pd.merge(total_df,key_target,on=key,how="left")
#         key_target=train_df['power'].groupby([train_df[key]]).std()
#         keys=key_target.keys().values
#         target=key_target.values
#         key_target=pd.DataFrame({key:keys,key+"_target_std":target})
#         total_df=pd.merge(total_df,key_target,on=key,how="left")

# #Extract all data according to 25% and 75%, low and high will be out_of_memory
# train_df=total_df[:len(train_df)]
# test_df=total_df[len(train_df):]
# del total_df
# print(f"total_feature_counts:{len(train_df.keys().values)}")

total_feature_counts:87


## Train and test Data Frames
extracing test_df and train_df from our total_df with new features

In [ ]:
main_df=total_df[:len(train_df)]
prediction_df=total_df[len(train_df):]

print(f"train set length : {len(train_df)}\ntest set length: {len(test_df)}")

train set length : 263160
test set length: 17136


# Gradient Boosting Ensemble Method
using LGBMRegressor for predictions

In [ ]:
from sklearn.model_selection import KFold #Import the k-fold cross validation function in the machine learning library
from lightgbm import LGBMRegressor #Import the integrated learning algorithm lightgbm
def RMSE(y_true,y_pred):
    return np.sqrt(np.mean((y_true-y_pred)**2))

In [ ]:
print("start fit.")
folds = 10 #Divide the data into 10 parts
y = main_df['power']
X = main_df.drop(['power'],axis=1)

train_RMSE=[]
valid_RMSE=[]
# Store the list of learned models
models = []

#Shuffle the data set randomly and divide it into folds
kf = KFold(n_splits=folds, shuffle=True, random_state=2023)

#Divide x_train into a training set and a verification set in a ratio of 9:1, and take out the subscripts
for train_index, valid_index in kf.split(X):

    #Get the data of the training set and validation set based on the subscripts
    x_train_cv = X.iloc[train_index]
    y_train_cv = y.iloc[train_index]
    x_valid_cv =X.iloc[valid_index]
    y_valid_cv = y.iloc[valid_index]

    #Call the LightGBM regression model and add parameters
    model = LGBMRegressor(colsample_bytree=0.6503468706312049,
              learning_rate=0.020010277043886332, max_bin=127,
              min_child_samples=8, n_estimators=1001,
              early_stopping_rounds = 100, #If the accuracy has not improved after training for 100 times, stop training.
              num_leaves=402,reg_alpha=0.01631050699150689,
              reg_lambda=0.01300300057057842,verbose=-1)


    #Use x_train_cv to train the model, and use x_train_cv and x_valid_cv to evaluate together
    model.fit(
        x_train_cv,
        y_train_cv,
        eval_set = [(x_train_cv, y_train_cv), (x_valid_cv, y_valid_cv)],
        #verbose = 100, #Iterate 100 times and output a result
    )

    #Predict the training set
    y_pred_train = model.predict(x_train_cv, num_iteration=model.best_iteration_)
    #Predict on the validation set
    y_pred_valid = model.predict(x_valid_cv, num_iteration=model.best_iteration_)

    train_rmse=RMSE(y_pred_train,y_train_cv)
    valid_rmse=RMSE(y_pred_valid,y_valid_cv)

    train_RMSE.append(train_rmse)
    valid_RMSE.append(valid_rmse)
    #Save model into list
    models.append(model)
    print(f"train_RMSE:{train_RMSE},valid_RMSE:{valid_RMSE}")

train_RMSE=np.array(train_RMSE)
valid_RMSE=np.array(valid_RMSE)

print(f"mean_train_RMSE:{np.mean(train_RMSE)}")
print(f"mean_valid_RMSE:{np.mean(valid_RMSE)}")

start fit.
train_RMSE:[0.3328398889321753],valid_RMSE:[0.38481594119123297]
train_RMSE:[0.3328398889321753, 0.33367642737094366],valid_RMSE:[0.38481594119123297, 0.3898513985594672]
train_RMSE:[0.3328398889321753, 0.33367642737094366, 0.33302245799146707],valid_RMSE:[0.38481594119123297, 0.3898513985594672, 0.3907355080851898]
train_RMSE:[0.3328398889321753, 0.33367642737094366, 0.33302245799146707, 0.33389790831093474],valid_RMSE:[0.38481594119123297, 0.3898513985594672, 0.3907355080851898, 0.3749267257585107]
train_RMSE:[0.3328398889321753, 0.33367642737094366, 0.33302245799146707, 0.33389790831093474, 0.33345142203302525],valid_RMSE:[0.38481594119123297, 0.3898513985594672, 0.3907355080851898, 0.3749267257585107, 0.3795972329863007]
train_RMSE:[0.3328398889321753, 0.33367642737094366, 0.33302245799146707, 0.33389790831093474, 0.33345142203302525, 0.33245138884184333],valid_RMSE:[0.38481594119123297, 0.3898513985594672, 0.3907355080851898, 0.3749267257585107, 0.3795972329863007, 0.38

In [ ]:
test_X = test_df.drop(['power'],axis=1).values
#Use each saved model to predict x_test once, and then take the average
preds_test = []

for model in models:

    pred = model.predict(test_X, num_iteration=model.best_iteration_)

    preds_test.append(pred)

#Convert the prediction results into np.array
preds_test_np = np.array(preds_test)
#Average the prediction results of each model by column
test_pred= preds_test_np.mean(axis=0)
test_pred=np.where(test_pred<=0,0,test_pred)
test_pred[:50]

In [ ]:
submission=pd.read_csv("sample_submission.csv")
submission['power']=test_pred
submission.to_csv("baseline.csv",index=None)
submission.head()

# Neural Network model
Using a Multi Layer model for predection

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
torch.manual_seed(42)

## Train and Test split

In [ ]:
train_test_ratio = 0.8
train_dataset = main_df[ : int(len(train_df)*train_test_ratio)]
test_dataset = main_df[int(len(train_df)*train_test_ratio) : ]

In [ ]:
X_train_tensor = torch.Tensor(train_dataset.drop(['power'],axis=1).values.astype(np.float32))
y_train_tensor = torch.Tensor(train_dataset['power'].values.astype(np.float32))

X_test_tensor = torch.Tensor(test_dataset.drop(['power'],axis=1).values.astype(np.float32))
y_test_tensor = torch.Tensor(test_dataset['power'].values.astype(np.float32))

In [ ]:
X_train_tensor.shape , y_train_tensor.shape

(torch.Size([210528, 7]), torch.Size([210528]))

In [ ]:
batch_size = 128

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import torch.nn as nn
import torch.optim as optim
device = "cpu"

In [ ]:
class RegressionModel(nn.Module):
    def __init__(self, input_features, hidden_units, output_size):
        super().__init__()
        self.linear_layer_stack = nn.Sequential(
            nn.Linear(in_features=input_features, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=4),
            nn.ReLU(),
            nn.Linear(in_features=4, out_features=output_size),
        )

    def forward(self, x):
        return self.linear_layer_stack(x)

In [ ]:
nn_model = RegressionModel(input_features= X_train_tensor.shape[1],
                    output_size=1,
                    hidden_units= 8).to(device)

In [ ]:
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.AdamW(nn_model.parameters(), lr=0.02)

In [ ]:
## try a single run on the model to check the output shapes
nn_model.train()
for inputs, targets in train_loader:
    output = nn_model(inputs)
    print("output shape = ", output.shape)
    targets = targets.unsqueeze(dim=1)
    print("targets shape = ", targets.shape)
    loss = criterion(output, targets)
    print("loss value = ", loss.item())
    break

output shape =  torch.Size([128, 1])
targets shape =  torch.Size([128, 1])
loss value =  1.1553386449813843


In [ ]:
# Training loop
num_epochs = 10

for epoch in range(num_epochs):
  print(f"Epoch: {epoch}\n-------")

  nn_model.train()
  ### Training
  train_loss = 0
  for batch, (inputs, targets) in enumerate(train_loader):
    targets = targets.unsqueeze(dim=1)
    # 1. Forward pass
    outputs = nn_model(inputs)
    # 2. Calculate loss (per batch)
    loss = torch.sqrt(criterion(outputs, targets))
    train_loss += loss # accumulatively add up the loss per epoch
    # 3. Optimizer zero grad
    optimizer.zero_grad()
    # 4. Loss backward
    loss.backward()
    # 5. Optimizer step
    optimizer.step()
    #if batch % 400 == 0:
      #print(f"Looked at {batch * len(inputs)}/{len(train_loader.dataset)} samples")

  # Divide total train loss by length of train dataloader (average loss per batch per epoch)
  train_loss /= len(train_loader)
  ### Testing
  # Setup variables for accumulatively adding up loss and accuracy
  test_loss = 0
  nn_model.eval()
  with torch.inference_mode():
    rmse_sum = 0.0
    num_samples = 0
    for inputs, targets in test_loader:
      targets = targets.unsqueeze(dim=1)
      # 1. Forward pass
      test_outputs = nn_model(inputs)
      # 2. Calculate loss (accumatively)
      test_loss += torch.sqrt(criterion(test_outputs, targets))

    # Divide total test loss by length of test dataloader (per epoch)
    test_loss /= len(test_loader)
  ## Print out what's happening
  print(f"\nTrain loss: {train_loss:.5f} | Test loss: {test_loss:.5f}")


Epoch: 0
-------

Train loss: 0.79628 | Test loss: 0.98649
Epoch: 1
-------

Train loss: 0.75656 | Test loss: 1.03925
Epoch: 2
-------

Train loss: 0.74396 | Test loss: 1.05232
Epoch: 3
-------

Train loss: 0.73867 | Test loss: 1.13611
Epoch: 4
-------

Train loss: 0.73675 | Test loss: 1.07861
Epoch: 5
-------

Train loss: 0.73654 | Test loss: 1.07169
Epoch: 6
-------

Train loss: 0.73681 | Test loss: 1.16946
Epoch: 7
-------

Train loss: 0.73753 | Test loss: 1.05498
Epoch: 8
-------

Train loss: 0.73740 | Test loss: 1.14926
Epoch: 9
-------

Train loss: 0.73784 | Test loss: 1.04795
